In [1]:
import cv2
from cv2 import aruco as ar
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from pythonosc.udp_client import SimpleUDPClient

In [2]:
ID_1 = 74
ID_2 = 84
ID_3 = 420
MARKER_SIZE = 5 #cm

#ancho y alto tienen que coincidir
#con la resolucion de las imagenes de calibracion
WIDTH = 640
HEIGHT = 480
FONT = cv2.FONT_HERSHEY_COMPLEX_SMALL
x = 0
y = 0

IP = '127.0.0.1'
PORT = 9007
MSG_POS = "/pos"
MSG_ROT = "/rot"

client = SimpleUDPClient(IP, PORT)

In [3]:
calib_path = "../calibration/results/"
CAMERA_MATRIX = np.loadtxt(calib_path + 'cameraMatrix.txt', delimiter=',')
CAMERA_DISTORSION = np.loadtxt(calib_path + 'cameraDistortion.txt', delimiter=',')

ARUCO_DICT = ar.getPredefinedDictionary(ar.DICT_ARUCO_ORIGINAL)
PARAMETERS = ar.DetectorParameters_create()

In [ ]:


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
while cap.isOpened():
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejected = ar.detectMarkers(
        image=gray,
        dictionary=ARUCO_DICT,
        parameters=PARAMETERS,
        cameraMatrix=CAMERA_MATRIX,
        distCoeff=CAMERA_DISTORSION)
    
    ids = np.array(ids)
    if ids.any():
        ret = ar.estimatePoseSingleMarkers(corners, MARKER_SIZE, CAMERA_MATRIX, CAMERA_DISTORSION)
        
        for i in range(ids.shape[0]):
            rvec, tvec = ret[0][i, 0,:], ret[1][i, 0,:]
            x = int(corners[i][0][0][0])
            y = int(corners[i][0][0][1])
        
            id_num = 0
            if ids[i] == ID_1:
                id_num = 1
                
            if ids[i] == ID_2:
                id_num = 2
                
            if ids[i] == ID_3:
                id_num = 3
            
#             client.send_message(f'/id/{id_num}{MSG_POS}', [tvec[0], tvec[1], tvec[2]])
            client.send_message(f'/id/{id_num}{MSG_ROT}', [rvec[0], rvec[1], rvec[2]])
            client.send_message(f'/id/{id_num}{MSG_POS}', [x, y])


            
            ar.drawDetectedMarkers(frame, corners)
            ar.drawAxis(frame, CAMERA_MATRIX, CAMERA_DISTORSION, rvec, tvec, 10)
            
            text_pos = f'[{id_num}]{ids[i]}=[x= {x}, y= {y}]'
#             text_pos = f'[{id_num}]{ids[i]}=[x={tvec[0]:.2f}, y={tvec[1]:.2f}, z={tvec[2]:.2f}]'
            text_rot = f'[{id_num}]{ids[i]}=[rx={rvec[0]:.2f}, ry={rvec[1]:.2f}, rz={rvec[2]:.2f}]'
            
            
            cv2.putText(frame,
                        text_pos,
                        (0, 50 * (i+1)),
                        FONT, 0.7, 
                        (0, 255, 0), 1)

            cv2.putText(frame,
                        text_rot,
                        (0, 50 * (i+2)),
                        FONT, 0.7, 
                        (0, 255, 0), 1)
            
            #DEBUG LOG
#             print(ids)
#             print(f'number of markers={ids.shape[0]}')
#             print(f'{ids[i]}=[x = {tvec[0]}, y = {tvec[1]}, z = {tvec[2]} marker = {i}]')
            

            
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()